In [1]:
import warnings
warnings.filterwarnings('ignore')

%config IPCompleter.greedy=True

In [2]:
import os
import openai
import pytesseract
from PIL import Image
import PyPDF2
import json
from dotenv import load_dotenv


### Load Environment Variables

In [37]:
load_dotenv()
openai.api_key= os.getenv('OPENAI_API_KEY') 

pytesseract.pytesseract.tesseract_cmd = os.getenv('OCR_PATH')  
basePath =os.getenv('BASE_PATH') 
loanDocsPath =basePath + "LoanDocs\\"
modelOutputPath =basePath + "GPTOutput\\"

In [38]:
print(basePath)

C:\Meetu\Meetu\Truist\GenAI\GenAIUseCases\LoanApprovalHelper\


In [39]:
print(loanDocsPath)

C:\Meetu\Meetu\Truist\GenAI\GenAIUseCases\LoanApprovalHelper\LoanDocs\


In [40]:
print(modelOutputPath)

C:\Meetu\Meetu\Truist\GenAI\GenAIUseCases\LoanApprovalHelper\GPTOutput\


## Functions

In [53]:
def readOCRData(path,fileName):
    # Path to the input image
    image_path=os.path.join( path, fileName )
    # Open the image using PIL (Python Imaging Library)
    image = Image.open(image_path)#'C:\Meetu\Meetu\Study\OpenAPI\OCR\GPTOCR\LoanDocs\CreditScore.jpg')

    # Perform OCR using pytesseract
    text = pytesseract.image_to_string(image)

    # Print the extracted text
    print(text) 
    return text

def writeToFile(path,fileToRead, textToWrite):
    fileName = path + fileToRead
    print (path)
    with open(fileName, 'w') as file:
        file.write(textToWrite)
        
def read_json_file(path,file_path):
    fileName = path + file_path
    with open(fileName, 'r') as file:
        data = json.load(file)
    return data

def readFromFile(path,fileToRead):
    fileName = path + fileToRead
    with open(fileName, encoding="utf8") as file:
        code = file.read()
    return code

def read_pdf(path,fileToRead):
    text = ""
    fileName = path + fileToRead
    with open(fileName, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        num_pages = len(pdf_reader.pages)
        for page_num in range(num_pages):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()
    return text

### Function to make call to OpenAI GPT 3.5 turbo model

In [8]:
 # Call OpenAI 
def getOpenAIResponse(systemPrompt, userPrompt, assistantPrompt):
    # Call OpenAI API to generate a description of the architecture
    
    response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
            {"role": "system", "content": systemPrompt},
            {"role": "user", "content": userPrompt},
          {"role": "assistant", "content": assistantPrompt},
        ],
        temperature=1,
    )
   
    #description = response.choices[0].text.strip()
    #print(response)
    return response.choices[0].message.content

## Perform OCR on Credit Score Image File

In [9]:
#OCR Data 
creditScoreData = readOCRData(loanDocsPath,'CreditScore.jpg')


CREDIT INFORMATION BUREAU (INDIA) LIMITED

IN ASSOCIATION WITH DUN & BRADSTREET AND TRANSUNION
CIBIL CONSUMER CREDIT INFORMATION REPORT

CONSUMER/BORROWER: PRABHU KABALI
MEMBER ID: BP00340001 (IMAGINARY BANK)
MEMBER REFERENCE NUMBER: BB-ENQO1

Date: 12-10-2006
TIME: 07:28:53
CONTROL NUMBER: 295,360,133

CONSUMER/BORROWER INFORMATION:
NAME: PRABHU KABALI

PERSONAL DETAILS TELEPHONE(S) IDENTIFICATION
DATE OF BIRTH: 16-08-1976 55332211 INCOME TAX ID: AMYFK0228P
VOTER ID:

GENDER: MALE 52951 Wee oat NG

55826
ADDRESS (ES):
ADDRESS DATE REPORTED
24/5 KANNAGI STREET, VINAYAGAPURAM AMBATTUR, CHENNAI 24-04-2006
TAMIL NADU 600053
24/5 KANNAGI STREET, VINAYAGAPURAM AMBATTUR, CHENNAI 02-01-2002

TAMIL NADU 600053

CIBIL TRANSUNION SCORE:
SCORE SCORING FACTORS
673 4: TEXT PF SCORING FACTOR GOES HERE, A REASON WHY THE CALCULATED SCORE IS WHAT IT IS.

2: TEXT PF SCORING FACTOR GOES HERE, POSSIBLY INFORMATION RELATED TO NUMBER OF OPEN ACCOUNTS OR
OVERALL CREDIT EXPOSURE.

SUMMARY:
ACCOUNT(S)
ACCOUNT 

In [10]:
writeToFile(modelOutputPath, "CreditScoreData.txt", creditScoreData)

C:\Meetu\Meetu\Truist\GenAI\GenAIUseCases\LoanApprovalHelper\GPTOutput\


## Classify Document Type

### List of pre defined Doc types

In [11]:
#Classification
DocTypes ="""
           1. CreditScore (CS)
           2. SalarySlip (SAL)
           3. EmployeeverficaitionLetter (EVL)
           4. LoanAgreement (LA)
           5. AddressProof (AP)
           6. PersonalIdentification (PI)
           7. Others (OTH)
        """

### Fetch Doc Type for the Credit Score document

In [12]:
###Classify
typeOfDoc= getOpenAIResponse("You are an efficient ECM expert in a big Indian bank", 
                                 f"""Please find what is the document type of the document  in double ticks ``{creditScoreData}``\n.
                                 Please use the document types from the list in triple ticks\n 
                                 ```{DocTypes}```.\n Please give the output in json format""", "")


In [13]:
print (typeOfDoc)

```json
{
    "Document Type": "CreditScore (CS)"
}
```


## Retrieve the Credit Score 

In [20]:
creditScore= getOpenAIResponse("You are an Loan Manager for a big Indian bank", 
                               "What is the credit score? Please give the output in json format ad only give the Credit Score with the key credit_score",
                               creditScoreData)


In [21]:
print(creditScore)
writeToFile(modelOutputPath,"CreditScoreResult.json",creditScore )
creditScorejson = read_json_file(ModelOutputPath,"CreditScoreResult.json")

{
  "credit_score": 673
}


In [24]:
CreditScoreForApplication=creditScorejson["credit_score"]
print(CreditScoreForApplication)

673


## Check weightage for Loan approval Process 
1. Read Rules defined in a text file
2. AI call to determine the weightage based on the credit score

In [41]:
rules =readFromFile(loanDocsPath,"RulesForLoanApproval.txt")
print(rules)

1.	Credit Score:
 	Credit Score: >701, Rating:Excellent,Weightage:15 points
	Credit Score: 501 to 700, Rating:Good,Weightage:10 points
	Credit Score: 301 to 500, Rating:Fair,Weightage:5 points
	Credit Score:<300, Rating:Poor,Weightage:-10 points
	preferred: Good	
2.	Income and Employment Verification:
	Stable job and consistent income: +10 points
	Unstable job or irregular income: -5 points
	preferred: Stable job and consistent income	
3.	Debt-to-Income Ratio (DTI):
	DTI below 30%: +12 points
	DTI between 30% and 40%: +6 points
	DTI above 40%: -10 points
	preferred: 35%
4.	Loan Amount and Collateral:
	Collateral offered (property value): +15 points
	Loan amount within 80% of collateral value: +10 points
	Loan amount exceeding collateral value: -10 points
	preferred: >70%
5.	Loan Purpose:
	Primary residence: +15 points
	Investment property: +5 points
	Vacant land or speculative projects: -10 points
	preferred: Primary Residence
6.	Regulatory Compliance:
	Adhering to all regulatory guide

In [26]:
creditScoreWeightage= getOpenAIResponse("You are an expert Data Analyst",
                                        f"""Please determine how much weightage with Credit Score of {creditScore}. 
                                        Rules for calculation of weightage is given in triple ticks. ```{rules} ``` .
                                        Please return only the weightage value.""", '')


In [27]:
print(creditScoreWeightage)

Based on the provided rules, the weightage for a Credit Score of 673 falls under the "Good" category, which has a weightage of 10 points. 

Therefore, the weightage value for a Credit Score of 673 is 10 points.


### Read Data from 
1. Employee Verification Details pdf file
2. Pay Slips


#### Employee Verfication Data

In [54]:
EmpVerficationData = read_pdf(loanDocsPath,'employmentletter.pdf')
writeToFile(basePath, "employmentletter.txt", EmpVerficationData)
print(EmpVerficationData) 

C:\Meetu\Meetu\Truist\GenAI\GenAIUseCases\LoanApprovalHelper\
3/15/20 23 
 
 
Mr. Mike Williams  
Century Apartment Complex 
25 Century Lane  
Springfield, MO 13243  
 
Subject: Letter of Employment Verification for Prabu Kabali  
 
 
Mr. Williams,  
 
 
Prabhu Kabali  has been employed as a Marketing Manager at Smith 
Industries since Jan 15th, 2010.  
Sally's annual salary as of 3/1/20 23 is 2,00,000  
Rupees . He works on a full -time basis.  
Please contact me at 1 800 555 5555 if I can be of any additional service.  
 
 
Sincerely,  
 
 
 
 
Bob Smith 
Smith  Industries  
123 Avenue North  
Springfield, MO 12345  


#### AI to Read requried details from the letter

In [55]:
EmploymentStatus= getOpenAIResponse("You are an expert Data Analyst",
                                        f"""Does the employee have a stable job from last 3 month
                                        based on the letter in  triple ticks. ```{EmpVerficationData} ``` .\n
                                        Format for output \n
                                        
                                        Years in Employment = 
                                        Is Full Time Employee (Y/N)=
                                        Start Date of Employment = 
                                        """, '')

In [56]:
print(EmploymentStatus)

Years in Employment = 10 years
Is Full Time Employee (Y/N) = Y
Start Date of Employment = Jan 15th, 2010


#### August Salary

In [58]:
payAugust2021 = read_pdf(loanDocsPath,'Payslip August 2021.pdf')
print(payAugust2021) 

 
Payslip  
 
ZooN oodle Private Limited  
 
1 Chennai Aminjikarai. No.4, West. Mada Street, Koyambedu,  
Aminjikarai, Chennai -. 600 107  
 
 
Date of Joining  : 2018 -06-23 
Pay Period  : August  2021  
Worked  Days  31 Employee  Name  : Prabhu Kabali  
Designation  : Marketing  Executive  
Department  : Marketing  
 
 
 
Earnings  
Basic Pay  Amount   
45000 Deductions  
Provident  Fund  Amount   
4800 
Incentive Pay   12450  Profe ssional Tax   900 
House Rent Allowance   20000 Education Cess   450 
Meal Allowance   2250     
 
Total Earnings   
79700   
Total Deductions   
6150  
  Net Pay  73550  
 
 
 
₹73550  
Rupees Seventy Three  Thousand 
Five Hundred  And Fifty . 
 
 
Employer  Signature  Employee  Signature  
 
 
 
 
 
 
 
This is system generated pay  slip and does not require a signature.  
 


In [59]:
AugustSalary= getOpenAIResponse("You are an expert Data Analyst",
                                        f"""What is salary and Month of the Salary of the employee from the PaySlip in triple ticks. ```{payAugust2021} ``` .\n
                                        Format for output \n
                                        
                                        Salary= 
                                        Month= .""", '')


In [60]:
print(AugustSalary)

``` 
Salary= ₹73,550   
Month= August 2021
```


#### September Salary

In [61]:
paySep2021 = read_pdf(loanDocsPath,'Payslip September 2021.pdf')
#writeToFile(basePath, "PayslipAugust2021.txt", payAugust2021)
print(paySep2021) 

 
Payslip  
 
ZooN oodle Private Limited  
 
1 Chennai Aminjikarai. No.4, West. Mada  Street,  Koyambedu,   
Aminjikarai,  Chennai -. 600 107  
 
 
Date of Joining  : 2018 -06-23 
Pay Period  : Sept  2021  
Worked  Days  30 Employee  Name  : Prabhu Kabali  
Designation  : Marketing  Executive  
Department  : Marketing  
 
 
 
Earnings  
Basic Pay  Amount   
45000 Deductions  
Provident  Fund  Amount   
4800 
Incentive Pay   13700  Profe ssional Tax   900 
House Rent Allowance   20000 Education Cess   450 
Meal Allowance   2250     
 
Total Earnings   
80950   
Total Deductions   
6150  
  Net Pay  74800  
 
 
 
₹74800  
Rupees Seventy Four  Thousand 
Eight  Hundred . 
 
 
Employer  Signature  Employee  Signature  
 
 
 
 
 
 
 
This is system generated pay  slip and does not require a signature . 
 


In [63]:
SepSalary= getOpenAIResponse("You are an expert Data Analyst",
                                        f"""What is salary and Month of the Salary of the employee from the PaySlip 
                                        in triple ticks. ```{paySep2021} ``` .\n
                                        Format for output \n
                                        
                                        Salary= 
                                        Month= .""", '')

In [64]:
print(SepSalary)

``` 
Salary= Rs. 74800 
Month= September 2021 
```


## Determine weightage for the loan Application based on the document
1. Credit Score
2. Employment Status
3. Pay Slips

In [65]:
userDataFromDocs =f"""
Credit Score : {CreditScoreForApplication}.\n
Employee Details : {EmploymentStatus}\n
Income: 
{SepSalary}\n
{AugustSalary}\n
Desired Loan Amount: 1500000 Rupees\n
Loan Purpose: Purchase of Home\n
Interest Rate for the Home Loan: 10%\n
Desired tenure: 20 years\n
User has been filling his tax returns regularly\n
User is looking for Flexible EMIs  \n
Value of house: 1600000 Rupees.\n

The user also has a scooter loan with monthly EMI of 3500 rupees.\n
Please calculate the EMI amount of the current loan in consideration . \n
Then use it to calculate the DTI. \n
Total Monthly EMI = EMI of the Current Loan + Other Loans
"""

weightageasPerDocs= getOpenAIResponse("You are an expert Data Analyst.", 
                             f"""Please determine the total weightage for a loan application with the critria 
                             included in double ticks below \n``{userDataFromDocs}``.\n 
                             Rules for calculation of weightage is given in triple ticks below.
                            ```{rules}```\n
                            Please share the steps used to calculate the weightage . 
                            Also include variance from the Preferred value and the final weightage.""","")

In [66]:
print(weightageasPerDocs)

First, let's calculate the EMI amount of the current loan in consideration:

EMI Calculation Formula:
EMI = [P x R x (1+R)^N] / [(1+R)^N - 1]
Where:
P = Loan amount (Principal) = 1500000 Rupees
R = Monthly interest rate = Annual interest rate / 12 = 10% / 12 = 0.0083333
N = Loan tenure in months = 20 years * 12 = 240 months

EMI = [1500000 x 0.0083333 x (1+0.0083333)^240] / [(1+0.0083333)^240 - 1]
EMI ≈ 14157.47 Rupees

Next, calculate the Total Monthly EMI by adding the EMI of the current loan and the EMI of other loans:
Total Monthly EMI = EMI of the Current Loan + Other Loans
Given that the EMI of the scooter loan is 3500 Rupees:
Total Monthly EMI = 14157.47 + 3500
Total Monthly EMI ≈ 17657.47 Rupees

Next, let's determine the weightage based on the criteria provided:

1. Credit Score:
Credit Score: 673 falls under the "Rating: Good" category
Weightage: 10 points

2. Income and Employment Verification:
Stable job and consistent income: +10 points

3. Debt-to-Income Ratio (DTI):
DTI 